In [1]:
import pandas as pd
import json
from time import sleep
from pymongo import MongoClient, errors

from icecream import ic

In [2]:
host = 'localhost'
port = 27017
db_name = 'oquant_runtime'
coll_name = '1234:group_prices'
strategy_id = '1'

client = MongoClient(host, port, serverSelectionTimeoutMS=5000)
db = client[db_name]
coll = db[coll_name]

all_groups = coll.find({'strategy_id': strategy_id})



In [3]:
def to_straddle_group(prices):
    def get_key_from_straddle_prices(price_data):
        if len(price_data) == 3:
            symbols = list(price_data.keys())
            (head, tail) = (symbols[1], symbols[2]) if symbols[1] > symbols[2] else (symbols[2], symbols[1])
            return f'{head}-{tail}' 
        return None
        
    def split(prices):
        all_data = {}    
        for dt, data in prices.items():
            if len(data) == 3:
                key = (get_key_from_straddle_prices(data))
                new_data = {'datetime': dt}
                new_data.update(data)
                if key in all_data:
                    all_data[key].append(new_data)
                else:
                    all_data[key] = [new_data]
        return all_data
    
    def to_df(all_data):
        all_df = {}
        for code, data in all_data.items():
            df = pd.DataFrame(data)
            df.set_index('datetime', inplace=True)
            all_df[code] = df
        return all_df
    
    all_data = split(prices)
    all_df = to_df(all_data)
    return all_df

In [4]:
for group in all_groups:
    prices = group['prices']
    prices = json.loads(prices)
    straddle_group = to_straddle_group(prices)
    
    break

In [5]:
profits = []
for code, df in straddle_group.items():
    symbols = code.split('-')
    df['total'] = df[symbols[0]] + df[symbols[1]]
    df['total'] = df['total'].bfill()
    
    profit = 10000 * (df['total'].iloc[0] - df['total'].iloc[-1])
    profit = round(10000 * (df['total'].iloc[0] - df['total'].iloc[-1]), 2)
    profits.append(profit)

In [6]:
straddle_group

{}

In [7]:
profits

[]

In [8]:
sum(profits)

0

In [9]:
prices

{'2020-06-17 09:30:00': {'510050.SH': 2.872, '10002535.SH': 0.012},
 '2020-06-17 09:31:00': {'510050.SH': 2.872, '10002535.SH': 0.0122},
 '2020-06-17 09:32:00': {'510050.SH': 2.873, '10002535.SH': 0.0125},
 '2020-06-17 09:33:00': {'510050.SH': 2.871, '10002535.SH': 0.0123},
 '2020-06-17 09:34:00': {'510050.SH': 2.87, '10002535.SH': 0.0124},
 '2020-06-17 09:35:00': {'510050.SH': 2.873, '10002535.SH': 0.0124},
 '2020-06-17 09:36:00': {'510050.SH': 2.873, '10002535.SH': 0.0126},
 '2020-06-17 09:37:00': {'510050.SH': 2.875, '10002535.SH': 0.0123},
 '2020-06-17 09:38:00': {'510050.SH': 2.875, '10002535.SH': 0.0124},
 '2020-06-17 09:39:00': {'510050.SH': 2.876, '10002535.SH': 0.0126},
 '2020-06-17 09:40:00': {'510050.SH': 2.877, '10002535.SH': 0.0128},
 '2020-06-17 09:41:00': {'510050.SH': 2.877, '10002535.SH': 0.0131},
 '2020-06-17 09:42:00': {'510050.SH': 2.879, '10002535.SH': 0.0135},
 '2020-06-17 09:43:00': {'510050.SH': 2.88, '10002535.SH': 0.0134},
 '2020-06-17 09:44:00': {'510050.SH':